<p style="text-align:right">
    项目名称：<b>人脸识别</b><br/>
设计：董相志<br/>
学号：220100<br/>
日期：2020.10<br/></p>

# 第一阶段：数据采集，构建训练集

本项目分为四个阶段：<br/><br/>
【1】第一阶段：数据采集，构建训练集。<br/>
【2】第二阶段：构建卷积网络，训练人脸识别模型。<br/>
【3】第三阶段：人脸相似度计算与比较。<br/>
【4】第四阶段：基于<b>One-Shot Learning</b>（单样本学习）实现人脸实时检测识别。<br/>

In [1]:
# 导入库
import cv2   
import dlib   # 用dlib做人脸检测
import numpy as np

如果需要安装dlib，请使用命令：<br/>
pip install cmake -i https://pypi.tuna.tsinghua.edu.cn/simple<br/>
pip install dlib -i https://pypi.tuna.tsinghua.edu.cn/simple<br/>
如果dlib安装提示错误，请注意观察，提示信息是否为：You need to install Visual Studio for C++<br/>
此时，你需要安装Visual Studio for C++ 2017或2019版本<br/>

### 还有一个更简洁的方法，下载dlib的whl文件，将其拷贝到安装目录，用pip命令实现本地安装
例如：假定你的工作环境为Python3.8。下载与3.8对应的dlib安装包。执行命令：<br/>
#### pip install dlib-19.19.0-cp38-cp38-win_amd64.whl

<font color='red'>运行下面的程序之前，应该首先在当前dataset目录下创建存放训练集和验证集图像的子目录：</font>

#### 目录结构：
<img src = "images/directory.PNG" />

#### 运行下面的程序格，每运行一次，可完成一人的数据采集

In [2]:
# 定义面部正面探测器
detector = dlib.get_frontal_face_detector()

# 打开摄像头或者打开视频文件
cap = cv2.VideoCapture(0)  #参数设为0，可以从摄像头实时采集头像
frame_count = 0  #帧计数
face_count = 0  #脸部计数
font = cv2.FONT_HERSHEY_SIMPLEX

# 循环读取每一帧，对每一帧做脸部检测，按ESC键循环结束
while True:  
    ret, frame = cap.read()  #从摄像头或者文件中读取一帧
    if (ret != True):
        print('没有捕获图像，数据采集结束或者检查摄像头是否工作正常！')
        break       
    frame_count += 1  # 帧计数
#     img_h, img_w, _ = np.shape(frame)  # 获取图像尺寸   
    detected = detector(frame, 1)  #对当前帧检测,参数1表示上采样1次
    faces = []   # 脸部图像列表  
    if len(detected) > 0:  #当前帧检测到人脸图像
        for i, d in enumerate(detected):  # 遍历
            face_count += 1  # 人脸计数
            # 脸部图像坐标与尺寸
            x1,y1,x2,y2,w,h = d.left(),d.top(),d.right()+1,d.bottom()+1,\
                              d.width(), d.height()
            # 脸部图像坐标
            face =  frame[y1:y2 + 1, x1:x2 + 1, :]
            # 将采集的图像自动分为训练集和验证集两部分，训练集占比75%，验证集占比25%
            if (frame_count % 4 !=0):
                #保存人脸图片到./dataset/train/目录,改变one目录，可保存采集的其他人图像
                # 用one、two、three、four做为目录名，也代表人名标签
                file_name = "./dataset/train/one/"+str(frame_count)+"_one"+str(i)+".jpg"
            else:
                #保存人脸图片到./dataset/valid/目录，改变one目录，可保存采集的其他人图像
                file_name = "./dataset/valid/one/"+str(frame_count)+"_one"+str(i)+".jpg"
            cv2.imwrite(file_name, face)  # 保存为文件
            # 绘制边界框
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"already get : {frame_count}  faces", \
                        (80, 80), font,1.2, (255, 0, 0), 3)
    # 显示单帧检测结果
    cv2.imshow("Face Detector", frame)
    # Esc键终止检测
    if cv2.waitKey(1) & 0xFF == 27:
        break
print('已经完成了 {0} 帧检测，共保存了 {1} 幅脸部图像'.format(frame_count, face_count))
cap.release()
cv2.destroyAllWindows()      

已经完成了 89 帧检测，共保存了 86 幅脸部图像


<font color = 'red'>需要特别强调的是，上面的数据采集程序，首次成功运行后，采集的数据默认保存在如下目录中：<br/></font>
<b>训练集目录：dataset/train/one<br/></b>
<b>验证集目录：dataset/valid/one<br/></b>
如果要采集第二个人的图像，请在重新运行上面程序格之前，将数据保存目录的代码中的<font color = 'red'>one修改为two。</font><br/>

In [ ]:
# 修改one--》two，注意，有四处地方需要修改
'''
if (frame_count % 4 !=0):
    file_name = "./dataset/train/two/"+str(frame_count)+"_two"+str(i)+".jpg"
else:
    file_name = "./dataset/valid/two/"+str(frame_count)+"_two"+str(i)+".jpg"
'''

本案例是一个通用程序，既可以从视频中采集图像，也可以从摄像头中采集图像。<br/>
以摄像头为例，大家分别站在摄像头前面，做出各种姿态与表情。完成数据采集工作。<br/>
数据采集完成后，应该分别检查目录one、two、three、four，看看是否单个目录中混杂了其他人的照片。这个工作是必要的，因为如果多个人一起站在摄像头前，有可能多个人的图像被采集到同一个目录里面。<br/>
熟悉采集的数据之后，可以进一步做人工标记，重新调整划分训练集与验证集比例等。<br/>

### 上面的工作结束后，恭喜您可以进行第二阶段的建模工作了。